# Setup

In [1]:
# !pip install pyngrok ngrok --force
# !ngrok authtoken "2AtiO9fAGfXvRDmPeuviD2BtA9L_512HFDFcGLVhqR5oPzVUT"
# from ngrok import ngrok

# listener = ngrok.connect(
#     # session configuration
#     addr="127.0.0.1:4040",
#     authtoken="2AtiO9fAGfXvRDmPeuviD2BtA9L_512HFDFcGLVhqR5oPzVUT",
#     session_metadata="Online in One Line",
#     # listener configuration
#     metadata="example listener metadata from python",
#     domain="fancy-basilisk-personally.ngrok-free.app",
#     schemes=["HTTPS"],
#     proto="http",
#     # module configuration
# )

In [2]:
!git clone https://github.com/facebookresearch/segment-anything-2.git
!cp -r segment-anything-2/* .

Cloning into 'segment-anything-2'...
remote: Enumerating objects: 510, done.
remote: Total 510 (delta 0), reused 0 (delta 0), pack-reused 510 (from 1)
Receiving objects: 100% (510/510), 88.46 MiB | 15.55 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [3]:
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.1/374.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
openai_key = userdata.get('yash_openai_api_key')

In [ ]:
import sys
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything-2.git'

  Cloning https://github.com/facebookresearch/segment-anything-2.git to /tmp/pip-req-build-0bpx5r9p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything-2.git /tmp/pip-req-build-0bpx5r9p
  Resolved https://github.com/facebookresearch/segment-anything-2.git to commit 7e1596c0b6462eb1d1ba7e1492430fed95023598


# Setting-up Classes

In [ ]:
import requests
from PIL import Image
import numpy as np
import torch
from transformers import pipeline

class ZeroShotObjectDetector:
  def __init__(self, labels):
    self.checkpoint = "google/owlv2-base-patch16-ensemble"
    self.detector = pipeline(model=self.checkpoint, task="zero-shot-object-detection", device="cuda")
    self.labels = labels

  def reset_labels(self, labels):
    self.labels = labels

  def apply_on_image(self, image):
    image = image.convert("RGB")
    predictions = self.detector(
        image,
        candidate_labels=self.labels,
    )
    return predictions

In [ ]:
import torch
from sam2.sam2_image_predictor import SAM2ImagePredictor
from PIL import Image
import numpy as np

class CustomYOLOSegment:
  def __init__(self):
    self.predictor = SAM2ImagePredictor.from_pretrained("facebook/sam2-hiera-tiny")

  def segment_image(self, input_box, image):
    input_box = np.array(input_box)
    with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
      self.predictor.set_image(np.array(image.convert("RGB")))
      masks, _, _ = self.predictor.predict(point_coords=None,
      point_labels=None,
      box=input_box[None, :],
      multimask_output=False,)
    return masks[0]

  def segment_array_of_bboxes(self, array_of_bboxes, image):
    masks = []
    for input_box in array_of_bboxes:
      mask = self.segment_image(input_box, image)
      masks.append(mask)
    return np.array(masks)

In [ ]:
import requests
import os

def download_image(url, filename):
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Write the content to a file
        with open(filename, 'wb') as file:
            file.write(response.content)
        print(f"Image successfully downloaded: {filename}")
    else:
        print(f"Failed to download image. Status code: {response.status_code}")

In [ ]:
from openai import OpenAI
import cv2

class FillWithDalleE:
  def __init__(self):
    self.client = OpenAI(api_key=openai_key)

  def edit_image(self, image, masks, prompt, pred_labels, filenames):
    image.save("image.png")
    rgb_image = np.array(image.convert("RGB"))
    urls = []
    for i in range(len(masks)):
      rgba_image = np.dstack((rgb_image, 255*(1-masks[i])))
      cv2.imwrite(f'mask_images_{i+1}.png', rgba_image)
      response = self.client.images.edit(
        model="dall-e-2",
        image=open("image.png", "rb"),
        mask=open(f'mask_images_{i+1}.png', "rb"),
        prompt=prompt+pred_labels[i],
        n=1,
        size="512x512"
      )
      image_url = response.data[0].url
      download_image(image_url, filenames[i])

# Making Inferences

In [ ]:
labels = ["shoe", "t-shirt", "cap"]
zsod = ZeroShotObjectDetector(labels)
cys = CustomYOLOSegment()
fwde = FillWithDalleE()

In [ ]:
image = Image.open('kid.png')
boxes = zsod.apply_on_image(image)
pred_labels = [box['label'] for box in boxes]
boxes = [[box['box'][l] for l in ['xmin', 'ymin', 'xmax', 'ymax']] for box in boxes]
masks = cys.segment_array_of_bboxes(boxes, image)
mask_means = [np.mean(mask) for mask in masks]
sorted_items = sorted(zip(mask_means, masks, pred_labels, boxes), key=lambda x: x[0], reverse=True)
sorted_mask_means, sorted_masks, sorted_pred_labels, sorted_boxes = zip(*sorted_items)
sorted_pred_labels = list(sorted_pred_labels)
sorted_boxes = list(sorted_boxes)
sorted_masks = np.stack(list(sorted_masks))
fwde.edit_image(image, sorted_masks[:4], "Suggests swift movement and athleticism, with the words \"NIKE\". This should be placed on a ", sorted_pred_labels, filenames=["img_"+str(i)+".png" for i in range(len(sorted_pred_labels))])

# Setting up Backend

In [ ]:
# !pip install flask-ngrok
!pip install flask-cloudflared

In [ ]:
from flask import Flask, request, jsonify, send_file
# from flask_ngrok import run_with_ngrok
from flask_cloudflared import run_with_cloudflared
import os
import base64

app = Flask(__name__)
run_with_cloudflared(app)

class Advertiser:
    def __init__(self, name, motive, objects_to_advertise):
        self.name = name
        self.motive = motive
        self.objects_to_advertise = objects_to_advertise

class Meme:
    def __init__(self, name, image_path, masks, pred_labels):
        self.name = name
        self.image_path = image_path
        self.masks = masks
        self.pred_labels = pred_labels

advertisers = []
memes = {}
unique_objects = set()

zsod = None
cys = CustomYOLOSegment()
fwde = FillWithDalleE()

def update_labels():
    global zsod
    zsod = ZeroShotObjectDetector(list(unique_objects))

@app.route('/advertiser', methods=['POST'])
def add_advertiser():
    data = request.json
    new_advertiser = Advertiser(data['name'], data['motive'], data['objects_to_advertise'])
    advertisers.append(new_advertiser)
    unique_objects.update(data['objects_to_advertise'])
    update_labels()
    return jsonify({"message": "Advertiser added successfully"}), 201

@app.route('/advertisers', methods=['GET'])
def get_advertisers():
    return jsonify([vars(adv) for adv in advertisers])

@app.route('/onboard_meme', methods=['POST'])
def onboard_meme():
    os.makedirs("uploads", exist_ok=True)
    if 'image' not in request.files:
        return jsonify({"error": "No image file provided"}), 400

    image = request.files['image']
    meme_name = request.form.get('name')

    if image.filename == '' or not meme_name:
        return jsonify({"error": "No selected image file or meme name not provided"}), 401

    if meme_name in memes:
        return jsonify({"error": "Meme name already exists"}), 402

    if image and zsod:
        image_path = os.path.join('uploads', f"{meme_name}_{image.filename}")
        image.save(image_path)

        img = Image.open(image_path)
        print("Inside", img.size)
        boxes = zsod.apply_on_image(img)
        pred_labels = [box['label'] for box in boxes]
        boxes = [[box['box'][l] for l in ['xmin', 'ymin', 'xmax', 'ymax']] for box in boxes]
        if len(boxes)==0:
          return jsonify({"error": "No objects to detect or image processing failed"}), 403
        masks = cys.segment_array_of_bboxes(boxes, img)
        mask_means = [np.mean(mask) for mask in masks]
        sorted_items = sorted(zip(mask_means, masks, pred_labels, boxes), key=lambda x: x[0], reverse=True)
        sorted_mask_means, sorted_masks, sorted_pred_labels, sorted_boxes = zip(*sorted_items)
        sorted_pred_labels = list(sorted_pred_labels)
        sorted_boxes = list(sorted_boxes)
        sorted_masks = np.stack(list(sorted_masks))

        new_meme = Meme(meme_name, image_path, sorted_masks[:4], sorted_pred_labels[:4])
        memes[meme_name] = new_meme

        return jsonify({"message": "Meme onboarded successfully", "meme_name": meme_name}), 201
    else:
        return jsonify({"error": "No objects to detect or image processing failed"}), 403

@app.route('/edit_meme/<meme_name>', methods=['POST'])
def edit_meme(meme_name):
    if meme_name not in memes:
        return jsonify({"error": "Meme not found"}), 404

    prompt = request.json.get('prompt', '')

    meme = memes[meme_name]
    img = Image.open(meme.image_path)
    edited_path = f"edited_{os.path.basename(meme.image_path)}"
    print(len(meme.masks))
    fwde.edit_image(img, meme.masks, prompt, meme.pred_labels, [edited_path]*len(meme.masks))
    print(edited_path)
    return jsonify({"message": "Meme edited successfully", "edited_image_path": edited_path}), 200

@app.route('/get_edited_meme/<path:filename>', methods=['GET'])
def get_edited_meme(filename):
    return send_file(filename, mimetype='image/jpeg')

@app.route('/get_meme_as_base64/<meme_name>', methods=['GET'])
def get_meme_as_base64(meme_name):
    if meme_name not in memes:
        return jsonify({"error": "Meme not found"}), 404

    meme = memes[meme_name]
    with open(meme.image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

    return jsonify({"image_base64": encoded_string})

@app.route('/get_meme_names', methods=['GET'])
def get_meme_names():
    return jsonify({"meme_names": list(memes.keys())})

In [ ]:
if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)